In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
from collections import deque

This notebook was prepared for exercises during [Lattice Practices 2023](https://indico.desy.de/event/40590/).

It starts with a background discussion which you are free to skip.

Here are the exercises:

Then we reduce the quantum-mechanical two-body scattering problem in 1D into a potential scattering problem. [ 2 small pen-and-paper exercises ]

We review the S matrix.

We impose periodic boundary conditions and the finite-volume quantization condition emerges. [ 2 pen-and-paper exercises and 3 questions to ponder ]

We at last reach numerical examples:

First we introduce a simple 1D quantum-mechanics solver, and check it against the free particle.
You have to implement the `free_spectrum` and `free_phase_shift`.

To transform the free particle finite-volume energies into scattering data you have to implement
 - `Luescher_x`
 - `S`
 - `cot_delta`
 - `Luescher_map`
   
but the results are essentially trivial.

Now a nontrivial example.  The $\delta$-function interaction.  Implement
 - The interaction itself
 - Can you improve the result?
 - What's going on with the bound state?

Finally, a mystery potential.  What can you say about it?  No peeking!

# Introduction

QCD is, of course, the theory of quarks and gluons.  At high energies the precision with which it can describe mesons and their properties is essential for understanding experimental signals, such as collider signals at the LHC.

In contrast, the low-energy sector of QCD–––the sector of protons, neutrons, and their interactions–––is much more poorly understood.  Or, rather, the precision with which we can extract numbers about nuclear physics from QCD remains much more limited–––there still exist theories of nuclear physics that are not quantitatively grounded in the Standard Model.

In the last decade or so the single-nucleon sector has come under excellent control.  The nucleon masses and the proton-neutron mass splitting have been determined at the physical point, in the continuum limit, for example.  A steady march towards precision single-nucleon matrix elements is under way.

Once you leave the single-nucleon sector, however, the field is much less developed.  However, multinucleon physics is interesting for a variety of experimental programs, such as low-energy underground BSM and neutrino experiments.  The targets in these experiments are not single nucleons but rather atomic nuclei.  So to interpret any signal from these experiments as constraints on new physics requires disentangling all the effects of many-body nuclear physics and QCD to pull out whatever may be new.

QCD is also, obviously, the foundation on top of which nuclear physics, in principle, is built.  However, making this connection quantitative has remained an outstanding problem since it became clear QCD was, in fact, the theory of the strong nuclear force.

The simplest multi-nucleon sector is the two nucleon sector.  Here, the simplest observables include the binding energies of any bound states and the scattering phase shifts.  We still lack a determination of these quantities from QCD at physical pion masses; at heavier pion masses there are still no calculations in the continuum limit.

Scattering is a real-time process.  It also formally relies on asymptotically separated states.  However, on the lattice, typically we have neither: we usually work in Euclidean time to have a well-defined probability measure and we work in a finite volume so that our calculations require a finite amount of RAM and a finite amount of execution time.  There is a no-go theorem due to Maiani and Testa ([PLB **245**, 585 (1990)](https://doi.org/10.1016/0370-2693(90)90695-3)) that seemingly prevented access to scattering observables.

However, Lüscher taught us (Commun. Math. Phys [104](https://doi.org/10.1007/BF01211589) and [105](https://doi.org/10.1007/BF01211097) (1986), [Nucl. Phys. B354 (1991) 531-578](https://doi.org/10.1016/0550-3213(91)90366-6), [Nucl. Phys. B364 (1991) 237-251](https://doi.org/10.1016/0550-3213(91)90584-K)) how to turn these seeming limitations to our advantage.  He provided a map from the finite volume spectrum (which can be determined in Euclidean time–––or real time, if you happen to have a method to do so!) to the infinite-volume phase shifts at those energies that appear in that finite volume.  Then, by changing the volume we work in we can change the allowed energies and fill in the phase shifts as a function of momentum.

Since those seminal works, there has been an enormous amount of theoretical progress, generalizing to moving frames, matrix elements, inelastic processes, the three-body case, and more.

See, for example,

 - Wiese [Nucl. Phys. Proc. Suppl. 9 (1989) 609-613](https://doi.org/10.1016/0920-5632(89)90171-0)
 - Gottlieb and Rummukainen [Nucl. Phys. Proc. Suppl 47 (1996) 819-822](https://doi.org/10.1016/0920-5632(96)00182-X)
 - Hansen and Sharpe [PRD 86 (2012) 016007](https://doi.org/10.1103/PhysRevD.86.016007)
 - Briceño and Davoudi [PRD 87 (2013) 9 094507](https://doi.org/10.1103/PhysRevD.87.094507) and [PRD 88 (2013) 9 094507](https://doi.org/10.1103/PhysRevD.88.094507)
 - Briceño, Davoudi, and Luu [PRD 88 (2013) 3 034502](https://doi.org/10.1103/PhysRevD.88.034502)
 - Briceño, Davoudi, Luu, and Savage [PRD 88 (2013) 11 114507](https://doi.org/10.1103/PhysRevD.88.114507) and [PRD 89 (2014) 7 074509](https://doi.org/10.1103/PhysRevD.89.074509)
 - Briceño, Hansen, and Walker-Loud [PRD 91 (2015) 3 034501](https://doi.org/10.1103/PhysRevD.91.034501)
 - Hansen and Sharpe [PRD 90 (2014) 11 116003](https://doi.org/10.1103/PhysRevD.90.116003), [PRD 92 (2015) 11 114509](https://doi.org/10.1103/PhysRevD.92.114509), [PRD 93 (2016) 014506](https://doi.org/10.1103/PhysRevD.93.014506), [PRD 93 (2016) 9 096006](https://doi.org/10.1103/PhysRevD.93.096006) and [erratum](https://doi.org/10.1103/PhysRevD.96.039901), [PRD 95 (2017) 3 034501](https://doi.org/10.1103/PhysRevD.95.034501)

Morever, there has been enormous success in *actually applying Lüscher's technique to learn about QCD*!  

**Here is a substantially incomplete list**.

The mesonic sector is by far the most sophisticated, and it's essentially impossible to cover the entire literature.

For recent progress, and a nearly-comprehensive-to-that-point review see

 - Briceño, Dudek, and Young [Rev.Mod.Phys. 90 (2018) 025001](https://doi.org/10.1103/RevModPhys.90.025001).

Once there are baryons, measuring correlators is much more painful and progress lags in comparison.

In the single-baryon sector, see

 - Lang and Verduci [PRD 87 (2013) 054502](https://doi.org/10.1103/PhysRevD.87.054502)
 - Kiratidis, Kamleh, Leinweber, Liu, Stokes, and Thomas [PRD 95 (2017) 074507](https://doi.org/10.1103/PhysRevD.95.074507)
 - Lang, Leskovec, Padmanath, and Prelovsek [PRD 95 (2017) 014510](https://doi.org/10.1103/PhysRevD.95.014510), [EPJ Web Conf. 175 (2018) 05004](https://doi.org/10.1051/epjconf/201817505004)

For two nucleon scattering, see

 - PACS-CS [PRD 84 (2011) 054506](https://doi.org/10.1103/PhysRevD.84.054506)
 - Yamazaki et al. [PRD 86 (2012) 074514](https://doi.org/10.1103/PhysRevD.86.074514)
 - NPLQCD [PRD 87 (2013) 3 034506](https://doi.org/10.1103/PhysRevD.87.034506)
 - Yamazaki et al. [PRD 92 (2015) 1 014501](https://doi.org/10.1103/PhysRevD.92.014501)
 - NPLQCD [PRD 92 (2015) 11 114512](https://doi.org/10.1103/PhysRevD.92.114512)
 - CalLat [PLB 765 (2017) 285-292](https://doi.org/10.1016/j.physletb.2016.12.024)
 - NPLQCD [PRD 96 (2017) 11 114510](https://doi.org/10.1103/PhysRevD.96.114510)
 
and for few-body matrix elements,

 - NPLQCD PRL 119 (2017) 6 [062002](https://doi.org/10.1103/PhysRevLett.119.062002) and [062003](https://doi.org/10.1103/PhysRevLett.119.062003)
 - NPLQCD [PRD 96 (2017) 5 054505](https://doi.org/10.1103/PhysRevD.96.054505)
 - NPLQCD [PRL 120 (2018) 15 152002](https://doi.org/10.1103/PhysRevLett.120.152002)

while for studies of the H dibaryon, see

 - NPLQCD [Mod. Phys. Lett A26 (2011) 2587-2595](https://doi.org/10.1142/S0217732311036978)
 - NPLQCD [PRL 106 (2011) 162002](https://doi.org/10.1103/PhysRevLett.106.162001)
 - Francis et al. [arXiv:1805.03966](http://arxiv.org/abs/arXiv:1805.03966)

In this notebook we will work through a simple problem in the same spirit: two interacting nonrelativistic quantum-mechanical particles in one spatial dimension.  

The approach will be as follows:
1. Set up the Hamiltonian of interest and review how putting it in a finite volume produces a quantization conditon.  We'll consider a ring of radius $L$.
2. Briefly review the S and T matrices.
3. Examine the infinite- and finite-volume cases.  Derive the quantization condition.  The quantization condition is the map from the finite-volume energies to scattering phase shift.

At that point, we will understand where Lüscher's formula comes from.  Then, we will apply it,

3. Discretize the finite-volume problem and extract its spectrum for a variety of radii.
4. Using the quantization condition, translate the energies found into phase shifts and compare with what we found in the infinite-volume case.

In lattice QCD we do not have access to the nuclear Hamiltonian.  However, we can nevertheless determine its spectrum and use the quantization condition to translate those results into phase shifts!

# One-Dimensional Quantum Mechanics Setup

Consider two distinguishable quantum mechanical particles that move in one infinite spatial dimension.  Let their interaction $V$ depend only on their separation.  Then, their Hamiltonian is given by

$$ \mathcal{H} = \frac{p_1^2}{2 m_1} + \frac{p_2^2}{2 m_2} + V(x_1-x_2) $$

We can switch to center-of-mass and relative coordinates.  Let

\begin{align}
    M &= m_1 + m_2                      &  \mu &= \frac{m_1 m_2}{M} \\
    X &= \frac{m_1 x_1 + m_2 x_2}{M}    &    x &= x_1 - x_2          \\
    K &= p_1+p_2                        &    k &= \frac{m_2 p_1 - m_1 p_2}{M}                \\
    U &= K / M                          &    u &= k/\mu
\end{align}

where the capital letters are center-of-mass variables, lower case letters are relative-coordinate variables, $M$ is mass, $X$ is position, $K$ is momentum, and $U$ is velocity.

Then it is easy to check the canonical commutation relations $[X,K]=[x,k]=i$ hold, and that $[X,k]=[x,K]=0$, so that we have completely decoupled the center-of-mass and relative motion.

### Exercise
Check that $[X,K]=[x,k]=i$ and that $[X,k]=[x,K]=0$, assuming the canonical commutation relations $[x_i,p_j]=i \delta_{ij}$

### Exercise
Check that the Hamiltonian may be rewritten in the new variables,

\begin{equation}
    \mathcal{H} = \frac{K^2}{2 M} + \frac{k^2}{2 \mu} + V(x)
\end{equation}

In these coordinates, it's clear that, as long as $V$ is even, there is a parity symmetry,

\begin{align}
    x &\rightarrow -x      &
    k &\rightarrow -k
\end{align}

because $[H, \mathcal{P}]=[V(x), \mathcal{P}]=0$ where $\mathcal{P}$ is the operator implementing the above transformation.

Moreover, the center of mass is completely free.  So, intereting physical infinite-volume observables can only be a function of the relative momentum $k$.

Since we have completely decoupled the center-of-mass degrees of freedom from the relative motion, we know the eigenfunctions can be expressed as products,

\begin{equation}
    \Psi(X,x) = e^{i K X} \psi_K(x)
\end{equation}

up to normalization.

# The S Matrix

Consider the scattering of two particles incident on one another in one dimension.  As the experimenter, we have the ability to control the momenta of the individual particles at the beginning of our scattering experiment.  That lets us set the initial $p_1$ and $p_2$ and thus the initial $K$ and $k$ arbitrarily.

After the particles interact, their momenta may be changed.  How this exactly happens we don't get to investigate microscopically–––we just have access to the asymptotic states.  What we do know, of course, is that center-of-mass momentum and energy are separately conserved.

As we showed above, we can trade a two-particle system for a one-particle system with a stationary potential (as long as we remember what we chose for $K$–––but we'll focus on the $K=0$ sector).

**We now adopt one-particle language for simplicity.**

The momenta after scattering is entirely fixed by kinematics, and is especially simple in 1 dimension.  

Conservation of energy and momentum imply that the only allowed final relative momenta are $\pm k$.

We can separately discuss cases when the particle is to the left or right of the potential,

\begin{align}
    x &> 0 && \text{``on the right"} \\
    x &< 0 && \text{``on the left".}
\end{align}

We can also identify cases where the particle is incident on on the potential or if it's heading away from the potential.  $k$ tells us which direction the particle is moving,

\begin{align}
    k &> 0 && \text{``moving to the right"} \\
    k &< 0 && \text{``moving to the left".}
\end{align}

Then, we have four kinds of states, two incoming states and two outgoing states.  If we agree that $k > 0$ and put in the signs explicitly then

\begin{align}
        \langle x |k; \text{incoming from the left} \rangle &\sim e^{+i k x}\theta(-x)
    &   \langle x |k; \text{outgoing to the right}  \rangle &\sim e^{+i k x}\theta(+x) \\
        \langle x |k; \text{outgoing to the left}   \rangle &\sim e^{-i k x}\theta(-x)
    &   \langle x |k; \text{incoming from the right}\rangle &\sim e^{-i k x}\theta(+x)
\end{align}

where the Heaviside-$\theta$ function tells us where the particle is in relation to the potential and the sign of $k$ tells us where it's heading.  We understand that the $\theta$-functions are really telling us the asymptotic region we're supposed to be looking in–––we don't really have access to the wavefunction inside the range of the potential.

The $S$-matrix is the map that takes some incoming states and produces some outgoing states.  If our initial state is given by

\begin{equation}
    \left|\Psi_i\right\rangle = \alpha \left|k; \text{incoming from the left}\right\rangle 
                              + \beta  \left|k; \text{incoming from the right}\right\rangle
\end{equation}

and the final state by

\begin{equation}
    \left|\Psi_f\right\rangle = a      \left|k; \text{outgoing to the right}\right\rangle
                              + b      \left|k; \text{outgoing to the left}\right\rangle.
\end{equation}

Then the $S$-matrix relates the coefficients,

\begin{equation}
    \left(\begin{matrix} a \\ b\end{matrix}\right) 
    = S(k)
    \left(\begin{matrix} \alpha \\ \beta \end{matrix}\right).
\end{equation}

Let's specialize to interactions with good parity.  We can symmetrize to states of even ($+1$) and odd ($-1$) parity,

\begin{align}
    \langle X, x \mid k; \text{incoming, parity }P \rangle &\sim \left(e^{+i k x} \theta(-x) + P e^{-i k x} \theta(+x) \right)/\sqrt{2} \\
    \langle X, x \mid k; \text{outgoing, parity }P \rangle &\sim \left(e^{+i k x} \theta(+x) + P e^{-i k x} \theta(-x) \right)/\sqrt{2}
\end{align}

and rewrite the asymptotic states

\begin{equation}
    \left|\Psi_i, k; \text{parity }P\right\rangle 
    = \alpha_\pm \left|k; \text{incoming, parity }P\right\rangle 
\end{equation}

and

\begin{equation}
    \left|\Psi_f, k; \text{parity }P\right\rangle 
    = a_\pm      \left|k; \text{outgoing, parity }P\right\rangle 
\end{equation}

Because of the symmetry of the Hamiltonian, we know the $S$ matrix doesn't mix these different parities.  So, we have diagonalized $S$-matrix,

\begin{align}
    a_+ &= S_+ \alpha_+
    &
    a_- &= S_- \alpha_-
\end{align}

In three spatial dimensions, the spiritually equivalent rewriting is in the basis of good angular momentum.

Since $S$ is unitary and the different channels don't talk to one another, each channel has its own independent scattering phase shift $\delta(k)$ that depends on the scattering momentum and determines the $S$ matrix  $S_\pm = \exp\left(2 i \delta_\pm(k)\right)$.

From general scattering theory, we know $S=1+iT$, and that $T$ admits a partial wave expansion.  As a reminder, in three spatial dimensions,

\begin{align}
    S_l(k) &= 1 + 2 i k f_l(k) & \text{or}&& f_l &= \frac{1}{k \cot \delta_l - i k}
\end{align}

The analogous result in 1 spatial dimension is

\begin{equation}
    T = \frac{k}{\mu} \frac{1}{\cot \delta(k) - i}
\end{equation}

In three dimensions $k \cot \delta_l$ is analytic in the energy and can be characterized by a Taylor series in $k^2$, the effective range expansion.

Similarly, in 1D $\cot \delta(k)$ enjoys an expansion in $k^2$.

# Quantization Condition for the Relative Wavefunction

For a time-independent state––which is another way to say a Hamiltonian eigenstate––where what's coming out of the scattering region reaches the periodic boundary and becomes the incoming wave, the relative wavefunction should, asymptotically, be some superposition of the incoming and outgoing states above.

Let $\psi(x) = \alpha\ \psi_\text{incoming}(x) + a\ \psi_\text{outgoing}(x)$.

In the $N=0$ sector, the periodicity condition reads

\begin{equation}
    \psi(x+ L \nu) = \psi(x)
\end{equation}

where $\psi$ is the relative wavefunction.  Let's let $\nu=1$ and set $x=-L/2$.  If $L$ is bigger than the range of the interaction, this is in the region where the wavefunctions look asymptotically like plane waves.

\begin{align}
    \psi(-L/2) &= \psi(L/2)
    \\
    \alpha e^{+i k (-L/2)} + a P e^{-i k (-L/2)} &= \alpha P e^{-i k (+L/2)} + a e^{+i k (+L/2)}
\end{align}

Massaging, one finds

\begin{equation}
    \frac{a}{\alpha} = e^{-i k L}
\end{equation}

If the two wavefunction components are related by the $S$ matrix then $a/\alpha = e^{2i\delta(k)}$ and

\begin{equation}
    e^{2 i \delta(k) + i k L} = 1
\end{equation}

That's it.  That's Lüscher's formula for 1D quantum mechanics!

Note that it contains no information about the interaction potential at all whatsoever.

In one dimension, to make the expression schematically match other dimensions, it is often written

\begin{equation}
    \frac{1}{k} \cot\delta(k) = \frac{L}{2\pi^2} S_1\left( 2 \mu E (L/2\pi)^2 \right)
    \qquad
    S_1(x) = -\pi \frac{\cot \pi \sqrt{x}}{\sqrt{x}}.
\end{equation}

### Exercise:

Show that this more complicated-looking expression is the same as the simple $e^{2i\delta(k) + i k L}=1$, when the on-shell condition $k^2 = 2\mu E$ is satisfied.

### Exercise:

I slighted you a bit.  When $P=+1$ the periodicity condition actually reduces to 0=0.  When the wavefunction is parity-even, of COURSE the wavefunction at $-L/2$ is the same as the wavefunction at $+L/2$.  Continuity across the boundary is guaranteed.  However, it's not guaranteed to be smooth.

Convince yourself that when $P=+1$, matching the derivative of the relative wavefunction reproduces the same quantization condition, and importantly, that matching the derivative doesn't overconstrain the $P=-1$ case!

### Questions

1.  What spatial symmetry/symmetries do the one-dimensional periodic boundary conditions have?  What spatial symmetry/symmetries does the infinite-volume $K=0$ sector of the Hamiltonian have?

2.  Think of periodic boundary conditions to three spatial dimensions.  What kind of functions solve them?  Do they have the symmetries of the analogous infinite-volume Hamiltonian?

3.  Do the symmetries of finite-volume QCD match infinite-volume QCD?  Is one symmetry group smaller than the other?  Does one contain the other?

As we discussed, the $S$-matrix relates asymptotic incoming states to asymptotic outgoing states.  This relation is fundamentally different from the relation given by the periodic boundary conditions.  Recall our earlier discussion: the $S$-matrix is the map that takes some incoming states and produces some outgoing states.  The boundary conditions select which, of all asymptotic states, fit just so.

# A Simple 1-Body Quantum Mechanics Solver

In this directory is `qm`, a solver for the quantum mechanics of one particle with $\mu=1$ and potential $V$ in a periodic box

In [ ]:
import qm

which we will introduce two simple cases.

<span style="color: red"> The units are such that µ=1! </span>

## Free Particle

### Expectations

Before we do anything numerical, let's understand what it is we hope to get out of our solver.

Q: What is the spectrum of a single free particle

$$ H = \frac{k^2}{2} $$

in a 1D box of length L with periodic boundary conditions?

Q: Which, states are degenerate, if any?

In [ ]:
def free_spectrum(n, L):
    # Excercise: remember 1D quantum mechanics?
    # Fill in this function so that for any positive integer n and real L
    # this returns the nth unique energy eigenvalue.
    return (2*np.pi*n/L)**2/2

In [ ]:
def free_phase_shift(k_squared):
    # Exercise:
    #
    # Fill in this function so that for any k^2 this returns
    # the phase shift δ(k)
    # for a free particle
    return np.zeros_like(k_squared)

### Setup

Let us just get familiar with our solver.  A set of Hamiltonians requires a potential which maps $x \rightarrow V(x)$,

In [ ]:
class free:

    def __init__(self):
        pass

    def __call__(self, lattice):
        # this has to evaluate for every x at once!
        return np.zeros_like(lattice.x)

V = free()
H = qm.Hamiltonian(V)

But we can only get something concrete if we provide a periodic lattice which has length L and is discretized into N sites.,

In [ ]:
L = 20
sites = 100
lattice = qm.Lattice(L, sites)

The lattice knows spatial coordinates

In [ ]:
lattice.x

and the lattice spacing

In [ ]:
lattice.dx

The free particle has no potential energy anywhere,

In [ ]:
V(lattice)

### Energies and Eigenfunctions

Now we can get the Hamiltonian for the lattice, which is sites × sites matrix.

In [ ]:
h = H(lattice)
print(h.shape)

You could now take this h and get its eigenvalues and so on, but the Hamiltonian itself provides a convenience function for getting the eigenvalues and eigenvectors.

In [ ]:
energies, wavefunctions = H.spectrum(lattice)

Let's check the resulting energies against your expectations.

In [ ]:
print(energies[:20])
print(free_spectrum(np.arange(11), L))

If they don't match as expected, go back!

### Plotting energies, wavefunctions.

`qm` provides a plotting utility to help you visualize the spectrum.

In [ ]:
fig, ax = plt.subplots()
states = 11 # How many states to visualize
qm.plot.spectrum(ax, lattice, energies, wavefunctions, states, potential=V)

### Finite-volume behavior

We can find the spectrum of the free Hamiltonian as a function of L.

In [ ]:
Ls = np.linspace(1,20)
nx = 100

states = 10

spectra = deque()

for L in Ls:
    e, v = H.spectrum(qm.Lattice(L, nx))
    spectra.append(e[:states])

# Rather than group the states by L, let's group them by eigenvalue number.
finite_volume_behavior = np.array(spectra).T

# Then we can track the finite-volume energies as a function of L
fig, ax = plt.subplots()
for state in finite_volume_behavior:
    ax.plot(Ls, state, marker='o', linestyle='none')

# You also provided the free spectrum, to which we can compare.
for state in range(states//2 + 1):
    L = np.linspace(1,20, 100)
    ax.plot(L, free_spectrum(state, L), color='black', zorder=-1)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim([1e-2,1e3])

ax.set_xlabel('L')
ax.set_ylabel('Eigenenergies')


### Lüscher Formalism

Recall the quantization condition

$$
    \frac{1}{k} \cot\delta(k) = \frac{L}{2\pi^2} S_1\left( 2 \mu E (L/2\pi)^2 \right)
    \qquad
    S_1(x) = -\pi \frac{\cot \pi \sqrt{x}}{\sqrt{x}}.
$$

where $E = k^2/2\mu$.

It is common to reexpress everything in terms of $x = 2\mu E (L/2\pi)^2 = (kL/2\pi)^2$.

$$
    \cot\delta(k(x)) = \frac{1}{\pi }\sqrt{\left(\frac{kL}{2\pi}\right)^2} S_1\left( 2 \mu E (L/2\pi)^2 \right) = \frac{1}{\pi} \sqrt{x} S_1(x)
$$

In [ ]:
def Luescher_x(E, L):
    # Exercise: given E and L, produce x.
    return 2*E*(L/(2*np.pi))**2 + 0.j

def S(x):
    # Exercise: implement S
    root_x = np.sqrt(x)
    return - np.pi / (root_x * np.tan(np.pi * root_x))

def cot_delta(E, L):
    # Exercise: implement cot(δ)
    x = Luescher_x(E, L)
    return 1 / np.pi * np.sqrt(x) * S(x)

# Unbelievably, numpy doesn't provide cotangent or arccot!
def cot(theta):
    return 1/np.tan(theta)

def arccot(x):
    # We want the principal value.
    # https://en.wikipedia.org/wiki/Inverse_trigonometric_functions#Principal_values
    # Therefore we need to mod into [0, π)
    
    arccot = np.arctan(1/x)
    return np.where(arccot.imag == 0, np.mod(arccot.real, np.pi), arccot)

# Finally, here's where the action is!
def Luescher_map(E, L):
    # Exercise: implement the map from (E, L) to k^2 and δ
    scattering_momentum_squared = 2*E + 0.j
    delta = arccot( cot_delta(E, L) )

    return scattering_momentum_squared, delta

In [ ]:
Ls = np.arange(1,5,0.25)
Nx = 100
trust = 10

fig, ax = plt.subplots()

ax.axhline(     0,   color='black', zorder=-1, linestyle=':')
ax.axhline(+np.pi/2, color='black', zorder=-1, linestyle=':')
ax.axhline(+np.pi,   color='black', zorder=-1, linestyle=':')

for L in Ls:

    energies, wavefunctions = H.spectrum(qm.Lattice(L, 100))
    
    # NOTE THE FOLLOWING DOES NOT DEPEND ON KNOWLEDGE OF THE POTENTIAL!
    # FINITE-VOLUME ENERGIES ONLY!
    E = energies[:trust]

    ksq, delta = Luescher_map(E,L)
    ax.plot(ksq, delta, "+")

ksq_max = 500
ksq = np.linspace(0, ksq_max, 100)
ax.plot(ksq, free_phase_shift(ksq), color='black', zorder=-1)

ax.set_xlabel(r'$k^2$')
ax.set_xlim([-0.05*ksq_max, 1.05*ksq_max])
ax.set_ylim([-0.1, np.pi+0.1])
ax.set_ylabel(r'$\delta(k)$')

Does it look right?  Is it as expected?

# δ-Function Interactions

## Setup

A delta-function interaction

$$ 
    V(x) = g \delta(x)
$$

has a bound state with binding energy $-g^2/2$ when $g<0$.

In [ ]:
class dirac_delta:

    def __init__(self, g):
        self.g = g
        self.binding = - g**2 / 2 if g < 0 else 0
    
    def __call__(self, lattice):
        # Exercise:
        # Given the points of the lattice x, construct the δ-function potential
        # with strength g.
        #
        # Does your solution really go to gδ(x) as we make the discretization finer?
        return self.g * (lattice.x == 0) / lattice.dx

Because of its simplicity, we can exactly compute the phase shifts from scattering off of this well.

Indeed, one can find in a variety of elementary quantum mechanics books that with $\mu=1$

$$ \tan 2\delta = -\frac{2gk}{k^2-g^2} $$

In [ ]:
def expectation(g, k_squared):
    return 0.5 * arccot( - (k_squared - g**2) / (2 * g * np.sqrt(k_squared)) )

<center><span style='color: red'> The goal is to reproduce this exact result only leveraging finite-volume spectra.</span></center>

In QCD we won't know the analogous result, but we *will* have access to the finite-volume spectra.  By finding another route to the scattering phase shift we can leverage the lattice to learn about scattering.

## Finite-Volume Spectrum

Let's look at an example finite-volume spectrum.

In [ ]:
g = -3
δ = dirac_delta(g)
print(f'Expect binding energy {δ.binding}')
H = qm.Hamiltonian(δ)
L = qm.Lattice(10, 100)
h = H(L)
fig, ax = plt.subplots()
qm.plot.spectrum(ax, L, *H.spectrum(L), 11, potential=δ)

ax.set_ylim([-5, 6])

Notice that the finite-volume ground state did not exactly match our expectations.

### Exercise:
How come?  Can you verify your answer numerically?

## Comparing to noninteracting energies

We can compare the energies as a function of $L$ to the free energies.

In [ ]:
Ls = np.linspace(1,20)
nx = 100

states = 10

spectra = deque()

for L in Ls:
    e, v = H.spectrum(qm.Lattice(L, nx))
    spectra.append(e[:states])

# Rather than group the states by L, let's group them by eigenvalue number.
finite_volume_behavior = np.array(spectra).T

# Then we can track the finite-volume energies as a function of L
fig, ax = plt.subplots()
for state in finite_volume_behavior:
    ax.plot(Ls, state, marker='o', linestyle='none')

# You also provided the free spectrum, to which we can compare.
for state in range(states//2 + 1):
    L = np.linspace(1,20, 100)
    ax.plot(L, free_spectrum(state, L), color='black', zorder=-1)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim([1e-2,1e3])

ax.set_xlabel('L')
ax.set_ylabel('Eigenenergies')

Some states seem to match for all $L$ while others deviate from the noninteracting energies.

### Question
What's the explanation?

## Nontrivial Scattering Data

OK, now for the reward for all this work!  Can we reproduce the known scattering phase shift using only the finite-volume energies?

In [ ]:
Ls = np.arange(1,10,1)
nx = 10
trust = 10

fig, ax = plt.subplots()
ax.axhline(     0,   color='black', zorder=-1, linestyle=':')
ax.axhline(+np.pi/2, color='black', zorder=-1, linestyle=':')
ax.axhline(+np.pi,   color='black', zorder=-1, linestyle=':')


spectra = deque()

for L in Ls:

    energies, wavefunctions = H.spectrum(qm.Lattice(L, nx))
    
    # NOTE THE FOLLOWING DOES NOT DEPEND ON KNOWLEDGE OF THE POTENTIAL!
    # FINITE-VOLUME ENERGIES ONLY!
    E = energies[:trust]
    spectra.append(E)
    ax.plot(*Luescher_map(E,L), "+")

# known answer:
ksq = np.linspace(0.1, 200, 1000)
ax.plot(ksq, expectation(g, ksq), color='black', zorder=-1)

ax.axhline(     0,   color='black', zorder=-1, linestyle=':')
ax.axhline(+np.pi/2, color='black', zorder=-1, linestyle=':')
ax.axhline(+np.pi,   color='black', zorder=-1, linestyle=':')
ax.axvline(2*δ.binding, color='black', zorder=-1)

ax.set_xlabel(r'$k^2$')
ax.set_xlim([-20, 200])
ax.set_ylim([-0.1, np.pi+0.1])
ax.set_ylabel(r'$\delta(k)$')

### Exercise
Does it match?  Can you improve it?

### Question
It seems like there are 2 distinct groups of states.  What's going on?

## The bound state

Let's look at the lowest energy level, which presumably corresponds to the bound state.

We know the exact value for the $\delta$-function potential, so let's compare.

In [ ]:
fig, ax = plt.subplots()

for L, E in zip(Ls, spectra):
    ax.plot((L,), (E[0],), 'o')

ax.axhline(δ.binding, color='black', zorder=-1, linestyle=':')
ax.set_xlabel('L')
ax.set_ylabel('Bound State Energy')

It appears that the ground state is deviating from the known result more and more as the box gets bigger and bigger.  That makes no sense---in bigger boxes the exponentially decay of a bound state should make the agreement better and better.

### Question:
What's the problem?  Can you verify your answer numerically?

# Mystery Potential

What if you're really blind to the potential?

In [ ]:
mystery = qm.examples.mystery_potential # Don't look!

### Exercise:
What can you tell me about this mystery interaction?